In [1]:
import cv2
import mediapipe as mp
import numpy as np
import winsound as ws

In [2]:
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [3]:
dots={'face':{},'lh':{},'rh':{},'pose':{}}
width=0; height=0; channel=0
face='face'; left='lh'; right='rh'; pose='pose'

In [4]:
def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [5]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [6]:
def collectData(results,dots):
    if results.left_hand_landmarks:
        for id,lm in enumerate(results.left_hand_landmarks.landmark):
            dots['lh'].update({str(id):{'x':int(lm.x*width),'y':int(lm.y*height),'z':int(lm.y*width)}})
    else:
        dots['lh']={}
    
    if results.right_hand_landmarks:
        for id,lm in enumerate(results.right_hand_landmarks.landmark):
            dots['rh'].update({str(id):{'x':int(lm.x*width),'y':int(lm.y*height),'z':int(lm.y*width)}})
    else:
        dots['lh']={}

    if results.pose_landmarks:
        for id,lm in enumerate(results.pose_landmarks.landmark):
            dots['pose'].update({str(id):{'x':int(lm.x*width),'y':int(lm.y*height),'z':int(lm.y*width)}})
    else:
        dots['pose']={}

    if results.face_landmarks:
        for id,lm in enumerate(results.face_landmarks.landmark):
            dots['face'].update({str(id):{'x':int(lm.x*width),'y':int(lm.y*height),'z':int(lm.y*width)}})
    else:
        dots['face']={}

In [7]:
def handler(func):
    def wrapper(*args):
        try:
            return func(*args)
        except:
            return False
    return wrapper

In [8]:
@handler
def intersect(dots,g1,p1,g2,p2):
    return abs(dots[g1][str(p1)]['x']-dots[g2][str(p2)]['x'])<20 and abs(dots[g1][str(p1)]['y']-dots[g2][str(p2)]['y'])<20 and abs(dots[g1][str(p1)]['z']-dots[g2][str(p2)]['z'])<20

In [9]:
@handler
def left_thumb_open(dots):
    if abs(dots['lh']['3']['x']-dots['lh']['5']['x'])>30:
        return True

@handler
def left_thumb_closed(dots):
    if abs(dots['lh']['3']['x']-dots['lh']['5']['x'])<30:
        return True

@handler
def left_fore_finger_up(dots):
    if dots['lh']['8']['y']<dots['lh']['5']['y']:
        return True

@handler
def left_fore_finger_down(dots):
    if dots['lh']['8']['y']>dots['lh']['5']['y']:
        return True

@handler
def left_middle_finger_up(dots):
    if dots['lh']['12']['y']<dots['lh']['9']['y']:
        return True

@handler
def left_middle_finger_down(dots):
    if dots['lh']['12']['y']>dots['lh']['9']['y']:
        return True

@handler
def left_ring_finger_up(dots):
    if dots['lh']['16']['y']<dots['lh']['13']['y']:
        return True

@handler
def left_ring_finger_down(dots):
    if dots['lh']['16']['y']>dots['lh']['13']['y']:
        return True

@handler
def left_small_finger_up(dots):
    if dots['lh']['20']['y']<dots['lh']['17']['y']:
        return True

@handler
def left_small_finger_down(dots):
    if dots['lh']['20']['y']>dots['lh']['17']['y']:
        return True

@handler
def left_thumb_up(dots):
    if abs(dots['lh']['3']['y']-dots['lh']['5']['y'])>30:
        return True

@handler
def left_thumb_down(dots):
    if abs(dots['lh']['3']['y']-dots['lh']['5']['y'])<30:
        return True

In [10]:
@handler
def right_thumb_open(dots):
    if abs(dots['rh']['3']['x']-dots['rh']['5']['x'])>30:
        return True

@handler
def right_thumb_closed(dots):
    if abs(dots['rh']['3']['x']-dots['rh']['5']['x'])<30:
        return True

@handler
def right_fore_finger_up(dots):
    if dots['rh']['8']['y']<dots['rh']['5']['y']:
        return True

@handler
def right_fore_finger_down(dots):
    if dots['rh']['8']['y']>dots['rh']['5']['y']:
        return True

@handler
def right_middle_finger_up(dots):
    if dots['rh']['12']['y']<dots['rh']['9']['y']:
        return True
    
@handler
def right_middle_finger_down(dots):
    if dots['rh']['12']['y']>dots['rh']['9']['y']:
        return True

@handler
def right_ring_finger_up(dots):
    if dots['rh']['16']['y']<dots['rh']['13']['y']:
        return True

@handler
def right_ring_finger_down(dots):
    if dots['rh']['16']['y']>dots['rh']['13']['y']:
        return True

@handler
def right_small_finger_up(dots):
    if dots['rh']['20']['y']<dots['rh']['17']['y']:
        return True

@handler
def right_small_finger_down(dots):
    if dots['rh']['20']['y']>dots['rh']['17']['y']:
        return True

@handler
def right_thumb_up(dots):
    if abs(dots['rh']['3']['y']-dots['rh']['5']['y'])>30:
        return True

@handler
def right_thumb_down(dots):
    if abs(dots['rh']['3']['y']-dots['rh']['5']['y'])<30:
        return True

In [11]:
@handler
def iLoveYou(dots):
    if (bool(dots['lh']) and left_thumb_open(dots) and left_fore_finger_up(dots) and left_middle_finger_down(dots) and left_ring_finger_down(dots) and left_small_finger_up(dots)) or (bool(dots['rh']) and right_thumb_open(dots) and right_fore_finger_up(dots) and right_middle_finger_down(dots) and right_ring_finger_down(dots) and right_small_finger_up(dots)):
        return True
    return False

In [12]:
@handler
def openRightFist(dots):
    if(bool(dots['rh']) and right_fore_finger_up(dots) and right_middle_finger_up(dots) and right_ring_finger_up(dots) and right_small_finger_up(dots)):
        return True
    return False

In [13]:
@handler
def openLeftFist(dots):
    if(bool(dots['lh']) and left_fore_finger_up(dots) and left_middle_finger_up(dots) and left_ring_finger_up(dots) and left_small_finger_up(dots)):
        return True
    return False

In [14]:
@handler
def house(dots):
    if openLeftFist(dots) and openRightFist(dots) and intersect(dots,left,12,right,12) and abs(dots[right]['0']['x']-dots[left]['0']['x'])>80 and abs(dots[left]['7']['x']-dots[left]['15']['x'])<=10 and abs(dots[right]['7']['x']-dots[right]['15']['x'])<=10:
        return True

In [15]:
@handler
def hello(dots):
    return openRightFist(dots) and intersect(dots,face,70,right,8)

In [16]:
@handler
def thanks(dots):
    return openLeftFist(dots) and openRightFist(dots) and intersect(dots,left,12,right,12) and intersect(dots,left,12,face,13) and abs(dots[left]['4']['x']-dots[right]['4']['x'])>80 

In [17]:
def printI(image,text:str,*args,color=(0,0,0),thickness=1):
    cv2.putText(image,text.format(args),(40,40),cv2.FONT_HERSHEY_TRIPLEX,thickness,color,thickness)

In [18]:
flow=[]

In [19]:
cap=cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,700)
holistic=mp_holistic.Holistic()
while cap.isOpened():
    ret,frame=cap.read()

    image,results=mediapipe_detection(frame,holistic)

    width,height,channel=image.shape

    collectData(results,dots)

    try:
        if iLoveYou(dots):
            printI(image,'ILoveYou')
            if flow==[] or flow[-1]!='I Love You':
                flow.append('I Love You')
        elif house(dots):
            printI(image,'House')
            if flow==[] or flow[-1]!='House':
                flow.append('House')
        elif hello(dots):
            printI(image,'Hello')
            if flow==[] or flow[-1]!='Hello':
                flow.append('Hello')
        elif thanks(dots):
            printI(image,'Thanks')
            if flow==[] or flow[-1]!='Thanks':
                flow.append('Thanks')
        else:
            printI(image,' ')
        
    
    except Exception as e:
        print('\n---------------------------')
        print(e)
        print('---------------------------\n')

    draw_landmarks(image,results)
    
    cv2.imshow('Action Recognition',image)

    key = cv2.waitKey(10)

    if key & 0xFF == ord('q'):
        ws.Beep(2000, 500)
        break
    elif key & 0xFF == ord('c'):
        print(flow)
        flow=[]
        ws.Beep(2000, 500)


cap.release()
cv2.destroyAllWindows()

['I Love You', 'House', 'I Love You', 'Thanks', 'Hello']


In [20]:
cap.release()
cv2.destroyAllWindows()